In [1]:
# coding=utf-8
import torch
from torch_geometric import transforms as T
torch.manual_seed(3407)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
### Step 1 定义模型和图数据

In [3]:
from src.model import GCN, DeepGCN, GAT
from src.tools import homo_data, split_homo_graph, Study, FocalLoss
import torch_geometric.transforms as T

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load and transform the data
transform = T.Compose([T.NormalizeFeatures(), T.ToDevice(device)])
data, id_mapping = homo_data(
    "./data/", transform=transform, fill_mode="stats", return_id_mapping=True
)

# Create a split based on parcel IDs (for example, using first 70% parcels for training)
# Assuming you want to split by some criterion, like parcel IDs
# 在主程序文件中
import torch
from torch_geometric.data import Data

# 假设你已经加载了data
# data, id_mapping = homo_data("./data/", return_id_mapping=True)

# 获取节点数量和类别数量
num_nodes = data.x.size(0)
num_classes = len(torch.unique(data.parcel_types))

# 创建分层采样的索引
train_idx, val_idx, test_idx = [], [], []

# 为每个类别进行分层采样以保持类别分布
for c in range(num_classes):
    # 获取该类别的所有节点索引
    idx = torch.nonzero(data.parcel_types == c).squeeze().tolist()
    # 确保idx是一个列表（处理只有一个样本的情况）
    if not isinstance(idx, list):
        idx = [idx]
    
    # 如果该类别没有样本，跳过
    if len(idx) == 0:
        continue
    
    # 打乱索引
    perm = torch.randperm(len(idx))
    idx = torch.tensor(idx)[perm]
    
    # 按照70%/15%/15%的比例划分（或者你想要的其他比例）
    n_train = int(0.7 * len(idx))
    n_val = int(0.15 * len(idx))
    
    train_idx.extend(idx[:n_train].tolist())
    val_idx.extend(idx[n_train:n_train+n_val].tolist())
    test_idx.extend(idx[n_train+n_val:].tolist())

# 创建掩码
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

# 添加掩码到数据对象
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

# 然后使用Study类进行训练

# 在训练前确保掩码和标签的形状匹配
if hasattr(data, 'parcel_types') and hasattr(data, 'train_mask'):
    if data.train_mask.size(0) != data.parcel_types.size(0):
        print(f"警告：掩码形状({data.train_mask.size(0)})与标签形状({data.parcel_types.size(0)})不匹配")
        # 裁剪掩码以匹配标签长度
        data.train_mask = data.train_mask[:data.parcel_types.size(0)]
        data.val_mask = data.val_mask[:data.parcel_types.size(0)]
        data.test_mask = data.test_mask[:data.parcel_types.size(0)]






# Or use RandomLinkSplit for link prediction tasks
# split = T.RandomLinkSplit(num_test=0.1, num_val=0.1)

警告：掩码形状(110836)与标签形状(110803)不匹配


In [4]:
# model = DeepGCN(data.num_features, 128, 64, 14).to(device)
# 获取类别数量


# 初始化模型
model = GCN(
    in_channels=data.num_features, 
    hidden_channels=128,
    out_channels=64,
    num_classes=num_classes
).to(device)
study = Study(model, data)  # 不需要提供split参数
# model = GAT(data.num_features, 128, 64, 5).to(device)



### Step 2 训练

In [5]:
CKPT_DIR = f"./ckpt/{model.name}/"
NUM_EPOCH = 100
# 现在可以训练模型
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = FocalLoss(alpha=0.25, gamma=2.0)
study.train(NUM_EPOCH, optimizer, save_dir=CKPT_DIR)

IndexError: The shape of the mask [110803] at index 0 does not match the shape of the indexed tensor [110836, 33] at index 0

### Step 3 测试

In [ ]:
# test on the model of the last epoch
# 带详细指标的测试
accuracy, predictions, metrics = study.test(data, return_metrics=True)

### Step 4 寻找最佳模型和最佳的topK
这里的`topK`是指病人和药品之间概率最`topK`大的边

In [ ]:
# study.find_best(CKPT_DIR, test_graph, NUM_EPOCH * 2, range(1, NUM_EPOCH + 1), range(1, 20))